In [2]:
import os
import gc
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm


# ===================== DATASET =====================
class CrowdDataset(Dataset):
    def __init__(self, img_paths, dens_paths, transform=None):
        self.img_paths = img_paths
        self.dens_paths = dens_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)

        dens = np.load(self.dens_paths[idx]).astype(np.float32)
        dens = torch.from_numpy(dens).unsqueeze(0)

        return img, dens


# ===================== TRANSFORMS =====================
transform = transforms.Compose([
    transforms.ToTensor(),
])


# ===================== PATHS =====================
img_dir = "processed_B/images"
dens_dir = "processed_B/density"

img_paths = [os.path.join(img_dir, f) for f in sorted(os.listdir(img_dir))]
dens_paths = [os.path.join(dens_dir, f) for f in sorted(os.listdir(dens_dir))]

dataset = CrowdDataset(img_paths, dens_paths, transform=transform)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# =====================================================
# CSRNet Training Script with Resume + Stats Saving
# =====================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

# ----------------- DEVICE -----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------- MODEL -----------------
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        # Frontend
        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        # Backend
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
        )

        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

# ----------------- DATASET & DATALOADER -----------------
# Assuming you have a Dataset object called "dataset"
# Example: dataset = CrowdDataset(img_paths, dens_paths, transform=transform)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# ----------------- CHECKPOINT PATHS -----------------
# from google.colab import drive
# drive.mount('/content/drive')
# drive_path = "/content/drive/MyDrive/CSRNet_training"
# os.makedirs(drive_path, exist_ok=True)

checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
# checkpoint_path = os.path.join(drive_path, "csrnet_checkpoint.pth")
stats_path = os.path.join(drive_path, "training_stats.npz")

# ----------------- MODEL + OPTIMIZER -----------------
model = CSRNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# ----------------- RESUME -----------------
start_epoch = 1
loss_history = []
gt_history = []
pred_history = []

if os.path.exists(checkpoint_path):
    print("🔄 Resuming training from checkpoint...")
    ckpt = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    start_epoch = ckpt["epoch"] + 1

    if os.path.exists(stats_path):
        data = np.load(stats_path, allow_pickle=True)
        loss_history = list(data["loss_history"])
        gt_history = list(data["gt_history"])
        pred_history = list(data["pred_history"])
else:
    print("🆕 Starting training from scratch!")

# ----------------- TRAIN LOOP -----------------
num_epochs = 10

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    total_gt = 0.0
    total_pred = 0.0

    pbar = tqdm(train_loader, total=len(train_loader))

    for img, dens in pbar:
        img = img.to(device)
        dens = dens.to(device)

        optimizer.zero_grad()
        pred = model(img)

        if pred.shape != dens.shape:
            pred = F.interpolate(pred, size=dens.shape[2:], mode="bilinear", align_corners=False)

        loss = F.mse_loss(pred, dens)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        epoch_loss += batch_loss

        gt_count = dens.sum().item()
        pred_count = pred.sum().item()

        total_gt += gt_count
        total_pred += pred_count

        pbar.set_description(
            f"Epoch {epoch}/{num_epochs} | batch_loss={batch_loss:.10f} | gt={gt_count:.2f} | pred={pred_count:.2f}"
        )

    # Save epoch stats
    loss_history.append(epoch_loss / len(train_loader))
    gt_history.append(total_gt)
    pred_history.append(total_pred)

    print(f"\n--- Epoch {epoch}/{num_epochs} | AvgLoss={loss_history[-1]:.10f} "
          f"| GT={total_gt:.2f} | Pred={total_pred:.2f} | Diff={abs(total_gt-total_pred):.2f} ---\n")

    # Save checkpoint & stats directly to Drive
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }, checkpoint_path)

    np.savez(stats_path,
             loss_history=np.array(loss_history, dtype=float),
             gt_history=np.array(gt_history, dtype=float),
             pred_history=np.array(pred_history, dtype=float))

    print("💾 Checkpoint + stats saved to Drive!\n")

NameError: name 'drive_path' is not defined